In [9]:
import os
import logging
from PIL import Image
import numpy as np

# Configure logging
logging.basicConfig(filename='file_processing.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
SOURCE_DIR_1 = "A"
SOURCE_DIR_2 = "B"
DEST_DIR = "labels"

In [11]:
# Create output directory if it doesn't exist
if not os.path.exists(DEST_DIR):
    os.makedirs(DEST_DIR)

files_a = {f for f in os.listdir(SOURCE_DIR_1) if f.endswith('.tif')}
files_b = {f for f in os.listdir(SOURCE_DIR_2) if f.endswith('.tif')}

# Ensure both directories contain the same files
common_files = files_a & files_b
if files_a != files_b:
    missing_in_a = files_b - files_a
    missing_in_b = files_a - files_b
    if missing_in_a:
        logging.error(f"Files missing in directory A: {missing_in_a}")
    if missing_in_b:
        logging.error(f"Files missing in directory B: {missing_in_b}")
    raise FileNotFoundError("Mismatch between files in directories A and B.")

for filename in common_files:
    file_a_path = os.path.join(SOURCE_DIR_1, filename)
    file_b_path = os.path.join(SOURCE_DIR_2, filename)
    try:
        # Open the images
        img_a = Image.open(file_a_path)
        img_b = Image.open(file_b_path)

        # Convert images to numpy arrays
        arr_a = np.array(img_a)
        arr_b = np.array(img_b)

        # Check if both images have the same shape
        if arr_a.shape != arr_b.shape:
            logging.error(f"Shape mismatch for file {filename}. Skipping this file.")
            continue

        # Process the images
        result = np.where((arr_a == 0) & (arr_b == 255) | (arr_a == 255) & (arr_b == 0), 255, 0)

        # Convert result back to image
        result_img = Image.fromarray(result.astype(np.uint8))

        # Save the output image
        output_file_path = os.path.join(DEST_DIR, filename)
        result_img.save(output_file_path)

        logging.info(f"Processed file {filename}. Output saved to {output_file_path}.")
    except Exception as e:
        logging.error(f"Failed to process file {filename}: {e}")